## Ddip User Guide 
### `%%dip` and `%autodip` --- Controlling Where the Code Are Executed

`%%dip` is a cell magic that says run the entire cell in either local, or remote DDP process group, or both namespaces.

`%autodip on" ` prepends `%%dip` to all subsequent cells, thus saving the repeated typing of `%%dip` in a stretch of cells all destined to be run in the DDP multiple processes.

**Combining `%%dip` and `%autodip` offers a cell-by-cell control of execution destination, thus interactively switching between,say exploring/fixing up raw data in local notebook, and creating training dataset, and training on the DDP group.**

In [1]:
%load_ext Ddip

In [2]:
%makedip -g all --verbose False

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-ippdpp_c-client.json


In [3]:
%%dip?

**By Default, `%%dip` alone implies `%%dip remote` execution, and to do so, it requires a DDP group be already created via the `%makedip` above**

To see outputs from all the processs, use `--see all`, otherwise, only Rank-0 output is displayed.

The following is run **only in all three DDP processes**, but not in the local notebook.

Try adding `-q` flag to see the difference in behavior --- it's only useful if user really does not need to see the transient output (e.g. progress bar when training?)

In [6]:
%%dip --see all
import time, random, os
for i in (range(3)):
    print(f"{i} ", end='')
    time.sleep(1)
print()
x = random.uniform(1,10)
print(f"Process [{os.getpid()}]: a uniform random number: {x}")

0 0 0 1 1 1 2 2 2 
Process [17491]: a uniform random number: 1.330240122772942

Process [17494]: a uniform random number: 3.2975983245233396

Process [17487]: a uniform random number: 1.2217514639879559


In [7]:
# back in local notebook, i doesn't exist
import os
print(f"Back in local process [{os.getpid()}], x is....")
x

Back in local process [17429], x is....


NameError: name 'x' is not defined

**To run a cell in both local and DDP processes (local first, then DDP) use `%%dip everywere`.**

In [8]:
%%dip everywhere
import os, random, time

# The libraries are imported to all namespaces: local notebook and DDP process group.

As a second example, ***can you tell which output is the local notebook process?***

In [9]:
%%dip everywhere --see all

import os, random
y = random.uniform(1,10)
print(f"Process [{os.getpid()}]: a uniform random number: {y}")
y

Process [17429]: a uniform random number: 7.1453557044632054


7.1453557044632054

Process [17487]: a uniform random number: 8.918987339531334
Process [17491]: a uniform random number: 1.3302454733974796
Process [17494]: a uniform random number: 6.535828163977395


Out[0:5]: 8.918987339531334

Out[1:5]: 1.3302454733974796

Out[2:5]: 6.535828163977395

### `%autodip`  **To run a series of cells under the same `%%dip` magic**


In [10]:
%autodip?

In [20]:
%autodip on

Auto Execution on DDP group: on, will run cell as %%dip


Next cell onward, will run in remote DDP process group.  By default only rank-0 process output is displayed.

In [23]:
dream = int(random.uniform(1,4))
print(f"Process [{os.getpid()}] dreaming {dream} seconds....")
while dream > 0:
    print(".", end='', flush=True)
    time.sleep(1)
    dream -= 1
print(f"\nProcess [{os.getpid()}] woke up...")

Process [17487] dreaming 1 seconds....
.
Process [17487] woke up...


**The default behavior of only streaming back output from rank 0 process** is useful when during distributed data parallel training, where rank 0 process aggregates and performs the all-reduce operation.  Often times only rank-0's output is of interest.

To see all outputs, for only one cell, use `%%dip --see all`, it'll be applied only to this cell.  Afterwards, back to whatever %autodip on settings it was before.

In [24]:
%%dip --see all
print(f"I am process [{os.getpid()}]")

I am process [17487]
I am process [17491]
I am process [17494]


In [26]:
# Back to only display rank-0 output
print(f"I am process [{os.getpid()}]")

I am process [17487]


#### **If `%%dip --see all` is desired for a series of cells, use the -a "args" flag to `%autodip`:**

In [28]:
%autodip -a "--see all" on

Auto Execution on DDP group: on, will run cell as %%dip --see all


In [29]:
print(f"I am process [{os.getpid()}], we are in automatic parallel execution mode.")

I am process [17487], we are in automatic parallel execution mode.
I am process [17491], we are in automatic parallel execution mode.
I am process [17494], we are in automatic parallel execution mode.


In [30]:
print(f"The time now is {time.time()}")

The time now is 1580405158.89015
The time now is 1580405158.8899994
The time now is 1580405158.8889928


#### While `%autodip` is `on`, a cell can ignore `%autodip` and change course to wherever, IF the first line begins with `%`.

**Say in the middle of `%autodip on`, let us do something in the local notebook, and push the result to DDP:**

In [38]:
%%dip local

s = f"Local process [{os.getpid()}] time was {time.time()}"

%dipush s

*Last cell overrides `%autodip`, now back to the last `%autodip -a "--see all" on` setting:*

In [39]:
print(f"Process [{os.getpid()}] : {time.time()} vs {s}")

Process [17487] : 1580405470.7578192 vs Local process [17429] time was 1580405469.8281991
Process [17491] : 1580405470.7588265 vs Local process [17429] time was 1580405469.8281991
Process [17494] : 1580405470.7586706 vs Local process [17429] time was 1580405469.8281991


Type `%autodip` alone in a cell shows its current on or off status

In [39]:
%autodip

Auto Execution on DDP group: on, will run cell as %%dip --see all


### In typical DDP training, `%autodip on` is sufficient, as we don't usually care about the output of non-Rank 0 processes.

#### To return to the local notebook namespaces for subsequent cells, turn off `%autodip`

In [42]:
%autodip off

Auto Execution on DDP group: Off
